# Trajectory inference and RNA velocity

## Introduction

In this notebook, we aim to accomplish 3 objectives:

1. Infer the trajectories of the CLL subclones using PAGA.
2. Give directionality to such trajectories with RNA velocity.
3. Find which genes change their expression as a funciton of pseudotime.

## Pre-processing

### Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sns
import scipy as sp
from scipy.interpolate import make_interp_spline, BSpline
import igraph
import scanpy as sc
import loompy
import scvelo as scv

In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

scanpy==1.4.5.post2 anndata==0.7.1 umap==0.3.10 numpy==1.17.5 scipy==1.4.1 pandas==0.25.3 scikit-learn==0.22.1 statsmodels==0.11.0 python-igraph==0.7.1


### Load data

In previous R markdown notebooks we save a Seurat list containing three fully demultiplexed, filtered, normalized, clustered and annotated Seurat objects. Each object corresponds to one chronic lymphocytic leukemia (CLL) patient (annotated as IGCG_012, ICGC_019 and ICGC_365). For each of them, we have serial peripheral blood samples at different stages of the diseases, starting at diagnosis end ending in Richter transformation.

Before working with PAGA, we need to convert the Seurat object to anndata object, which we will accomplish with the [`sceasy` package](https://github.com/cellgeni/sceasy):

In [5]:
%load_ext rpy2.ipython

ModuleNotFoundError: No module named 'tzlocal'

In [4]:
%R%
mtcars

UsageError: Line magic function `%R%` not found.


In [ ]:
%R%
reticulate::use_conda_env("/opt/anaconda3/envs/scanpy3")
richter_l_seurat <- readRDS("results/R_objects/richter_seurat_patients_list_clustered.rds")
for (seurat in richter_l_seurat) {
    sceasy::convertFormat(
        seurat, 
        from = "seurat", 
        to = "anndata", 
        outFile = "results/anndata_objects/"richter_seurat_patients_list_clustered")
}

reticulate::use_virtualenv("/opt/anaconda3/envs/scanpy3/")
test888 <- readRDS("~/Google Drive/single_cell/PhD/B_cell_atlas/RICHTER/current/results/R_objects/richter_seurat_patients_list_clustered.rds")
test888 <- test888[[1]]
sceasy::convertFormat(test888, from = "seurat", to = "anndata", outFile = "./test888.h5ad")
    

## RNA velocity

In [4]:
files_012 = ["data/BCLLATLAS_10/ICGC_012_01/velocyto/ICGC_012_01.loom",
         "data/BCLLATLAS_10/ICGC_012_02/velocyto/ICGC_012_02.loom"]
output_name_012 = "data/BCLLATLAS_10/ICGC_012_01/velocyto/ICGC_012_01_02.loom"
loompy.combine(files_012, output_file = output_name_012, key = "Accession")

In [5]:
files_019 = ["data/BCLLATLAS_10/ICGC_019_01/velocyto/ICGC_019_01.loom",
         "data/BCLLATLAS_10/ICGC_019_02/velocyto/ICGC_019_02.loom"]
output_name_019 = "data/BCLLATLAS_10/ICGC_019_01/velocyto/ICGC_019_01_02.loom"
loompy.combine(files_019, output_file = output_name_019, key = "Accession")

In [6]:
output_name_365 = "data/BCLLATLAS_10/ICGC_365/velocyto/ICGC_365.loom"
adata_scvelo_012 = sc.read_loom(output_name_012)
adata_scvelo_019 = sc.read_loom(output_name_019)
adata_scvelo_365 = sc.read_loom(output_name_365)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [8]:
adata_scvelo = [adata_scvelo_012, adata_scvelo_019, adata_scvelo_365]

In [9]:
adata_scvelo

[AnnData object with n_obs × n_vars = 13036 × 33538 
     obs: 'Clusters', '_X', '_Y'
     var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
     layers: 'matrix', 'ambiguous', 'spliced', 'unspliced',
 AnnData object with n_obs × n_vars = 25320 × 33538 
     obs: 'Clusters', '_X', '_Y'
     var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
     layers: 'matrix', 'ambiguous', 'spliced', 'unspliced',
 AnnData object with n_obs × n_vars = 5524 × 33538 
     obs: 'Clusters', '_X', '_Y'
     var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
     layers: 'matrix', 'ambiguous', 'spliced', 'unspliced']